# Install libs 

In [1]:
!pip install rasterio
!pip install geopandas
!pip install geemap
!pip install geedim
!pip install -U scikit-image
!pip install whitebox
!pip install asf-search
!pip install folium

# Import Libs

In [2]:
from osgeo import gdal 
import ee
import os
import math
import shutil
import folium
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
import tempfile
import asf_search as asf
import geemap
import geedim

# gdal need to be installed in the environment
# ESA SNAP also need to be installed 

# Inputs

## Set Folders

In [26]:
# Set study area number (between 1 and 9)
i = 1

# Set the path of the shapefile of the study area
shapefile_path = f'C:/Users/gabro/OneDrive/Área de Trabalho/artigo_tese/posicao_areas_estudo/{i}/{i}_wgs84.shp'

# Set the path the DEM and its water bodies mask (DEM its bigger then the ROI because of the hydrologic features extraction)
dem_path = f"D:/thesis_areas/area_0{i}/new_corrected_dem.tif"
waterbodies_path = f"D:/thesis_areas/area_0{i}/glo-30_wbm.tif"

# Output folders for the features and auxiliary files
features_path = f"D:/thesis_areas/area_0{i}/features" 
aux_path = f'D:/thesis_areas/area_0{i}/auxiliar'          

# Create output dirs 
if not os.path.exists(features_path):
    os.mkdir(features_path)

if not os.path.exists(aux_path):
    os.mkdir(aux_path)

## Set Dates

In [4]:
# Sentinel-2 dates 
START_DATE = '2018-07-01'
END_DATE = '2023-12-31'

# Sentinel-1 dates
start_date = '2014-10-03'
end_date = '2023-12-31'

# For the Palsar yearly mosaics the entire time-series was used

# Functions

## Temporal

### Cloud masking

In [11]:
# Cloud masking parameters
CLOUD_FILTER = 10
CLD_PRB_THRESH = 30
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 150

# Get Sentinel-2 collection joined with S2 Cloud Probability images
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

# Add cloud components
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

# Add shadow components
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

# Calculate cloud/shadow mask
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

# Apply cloud/shadow mask
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

def display_cloud_layers(col, aoi):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = aoi.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

### Daily Mean, daily mosaics

In [12]:
# FUNCTIONS DEFINITION - Calculate EVI, NDWI and time-series metrics

def daily_images_mean(collection):
    # This function computes the mean of images that fall on the same day and preserves the original image names.

    # Get a list of distinct dates in the collection.
    distinct_dates = ee.List(collection.aggregate_array('system:time_start')).map(lambda time: ee.Date(time).format('YYYY-MM-dd')).distinct()

    # Define a function that takes a date and filters the collection to include only images from that date.
    def filter_date(date):
        date = ee.Date(date)
        filtered_images = collection.filterDate(date, date.advance(1, 'day'))

        # Get the original IDs of the images being combined.
        original_ids = filtered_images.aggregate_array('system:index')

        # Combine the images by taking the mean of all images in this date.
        mean_image = filtered_images.mean().set({
            'system:index':date.format('YYYY-MM-dd'),
            'system:time_start': date.millis(),
            'date': date.format('YYYY-MM-dd'),
            'original_ids': original_ids
        })
        return mean_image

    # Map the function over the list of dates to get a new image collection where each image is the mean of images from the same day.
    mean_daily_collection = ee.ImageCollection.fromImages(distinct_dates.map(filter_date))

    return mean_daily_collection

In [13]:
def daily_mosaics(collection):
        # This function computes the mean of images that fall on the same day and preserves the original image names.
    
        # Get a list of distinct dates in the collection.
        distinct_dates = ee.List(collection.aggregate_array('system:time_start')).map(lambda time: ee.Date(time).format('YYYY-MM-dd')).distinct()
    
        # Define a function that takes a date and filters the collection to include only images from that date.
        def filter_date(date):
            date = ee.Date(date)
            filtered_images = collection.filterDate(date, date.advance(1, 'day'))
            
            # Get the original IDs of the images being combined.
            original_ids = filtered_images.aggregate_array('system:index')
            
            # Combine the images by taking the mean of all images in this date.
            mosaic = filtered_images.mosaic().set({
                'system:index':date.format('YYYY-MM-dd'),
                'system:time_start': date.millis(),
                'date': date.format('YYYY-MM-dd'),
                'original_ids': original_ids
            })
            return mosaic
    
        # Map the function over the list of dates to get a new image collection where each image is the mean of images from the same day.
        daily_mosaics_collection = ee.ImageCollection.fromImages(distinct_dates.map(filter_date))
    
        return daily_mosaics_collection

### NDVI/EVI Indexes

In [14]:
def getEVI(image):
    # Compute the EVI using an expression.
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8').divide(10000),
            'RED': image.select('B4').divide(10000),
            'BLUE': image.select('B2').divide(10000)
        }).rename("EVI")

    image = image.addBands(EVI)

    return(image)

def getNDWI(image):
    # Compute NDWI
    NDWI =  image.normalizedDifference(['B3','B8']).rename("NDWI")
    image = image.addBands(NDWI)
   
    return(image)

### Composites

In [15]:
def extract_features_gee(col):

    # Getting band names (assuming that are equal for all images)
    bandnames = col.first().bandNames().getInfo()

    # Calculate mean, std and percentiles
    col_mean = col.mean().rename([f'{bandname}_mean' for bandname in bandnames])
    col_std = col.reduce(ee.Reducer.stdDev()).rename([f'{bandname}_std' for bandname in bandnames])
    col_percentiles = col.reduce(ee.Reducer.percentile([5, 25, 50, 75, 99], ['05','25','50','75','99']))

    # Stack them and sort alphabetically
    col_features = col_mean.addBands([col_std, col_percentiles])
    sorted_names = col_features.bandNames().sort()
    col_features = col_features.select(sorted_names)

    return col_features

### Other

In [ ]:
# Filters out Sentinel-2 images with different band set
# Some studies areas had a few scenes with inconsistent band names, so they were removed
def filter_images_by_band_names(collection, required_bands):
    """
    Filters a GEE ImageCollection to include only images with exact band names matching the required list.

    Args:
        collection (ee.ImageCollection): The input GEE image collection.
        required_bands (list): The list of band names to match, in exact order.

    Returns:
        ee.ImageCollection: The filtered image collection.
    """
    # Convert the required bands list to an ee.List for GEE compatibility
    required_bands_ee = ee.List(required_bands)

    # Define a function to check if the image has the required bands
    def has_required_bands(image):
        # Compare the image's band names with the required bands
        is_matching = image.bandNames().equals(required_bands_ee)
        # Add a property to indicate if the image matches
        return image.set('isMatching', is_matching)

    # Map the function over the collection
    mapped_collection = collection.map(has_required_bands)

    # Filter images that have 'isMatching' set to True
    filtered_collection = mapped_collection.filterMetadata('isMatching', 'equals', True)

    return filtered_collection

# Define the required band names in the exact order
required_bands = [
    "B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B11", "B12",
    "AOT", "WVP", "SCL", "TCI_R", "TCI_G", "TCI_B", "MSK_CLDPRB", "MSK_SNWPRB",
    "QA10", "QA20", "QA60", "probability", "clouds", "dark_pixels", 
    "cloud_transform", "shadows", "cloudmask"
]

In [16]:
# Function to clip raster with a shapefile 
# a buffer (in pixels) can be set for padding the image 
# this is useful for applying denoise on a clipped image instead of the entire one,
# while avoiding border effects
def clip_image_with_shapefile(image_path, shapefile_path, output_path, nr_buffer_pixels):

    # Load the shapefile using geopandas
    gdf = gpd.read_file(shapefile_path)

    # Load the image using rasterio
    with rasterio.open(image_path) as src:
        # Convert the shapefile's CRS to match the image's CRS
        gdf = gdf.to_crs(src.crs)

        # Buffer the shapefile's geometry by the specified number of pixels
        gdf['geometry'] = gdf.buffer(src.res[0] * nr_buffer_pixels)

        # Use the buffered geometry to mask (or clip) the image
        out_image, out_transform = mask(src, gdf.geometry, crop=True)

        # Write the clipped image to a new file
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)


# Auxiliary function to get image metadata (used on the align_raster)
def get_raster_info(raster_path):
    with rasterio.open(raster_path) as src:
        # Get the bounding box (left, bottom, right, top)
        bounds = src.bounds
        # Get the x and y resolution
        xres, yres = src.res
        # Get the spatial reference system (SRS)
        srs = src.crs.to_string()

    return bounds, xres, yres, srs

# Function to align target image to a reference one
def align_raster(target_path, ref_path, resampling, dtype, output_path):
    
    #possible dtypes : Byte, Int8, UInt16, Int16, UInt32, Int32, UInt64, Int64, Float32, Float64, CInt16, CInt32, CFloat32 or CFloat64
    
    raster_a = target_path
    raster_b = ref_path
    output_a = output_path

    # Get raster info
    bounds_b, xres_b, yres_b, srs_b = get_raster_info(raster_b)

    # Construct the gdalwarp command
    cmd = (
        f'gdalwarp -t_srs {srs_b} -te {bounds_b.left} {bounds_b.bottom} {bounds_b.right} {bounds_b.top} '
        f'-tr {xres_b} {yres_b} "{raster_a}" "{output_a}" -r {resampling} -ot {dtype} -overwrite')
    
    print(cmd)
    # Execute the command
    flag = os.system(cmd)

    if flag != 0:
        print('Error executing gdal_warp.')

In [17]:
#function to mask the noisy edges of the images
def mask_edges(image):
    
    # Get the valid pixel mask
    valid_mask = image.mask()
    
    # Iteratively erode the valid mask (adjust iterations as needed)
    eroded_mask = valid_mask.focal_min(iterations=50) # Adjust the number of iterations
    
    # Apply the eroded mask to the image
    return image.updateMask(eroded_mask)

## Morphometric

### TPI, TRI, Slope

In [18]:
from osgeo import gdal 
import rasterio
from scipy.ndimage import generic_filter
from skimage.morphology import disk
import numpy as np
import tempfile
import os

def circular_mean_filter(input_file, output_file, radius):
    """
    Apply a circular mean filter to a GeoTIFF file using a specified radius.

    Parameters:
    - input_file: str, path to the input GeoTIFF file.
    - output_file: str, path to save the filtered GeoTIFF file.
    - radius: int, radius of the disk for the circular footprint. If radius = 0, no filtering is applied.
    """
    # Open the input GeoTIFF file using rasterio
    with rasterio.open(input_file) as src:
        profile = src.profile  # Get the metadata for writing later

        # Read the entire raster data into memory and apply filtering
        if radius > 0:
            footprint = disk(radius, strict_radius=True)  # Create a circular footprint
            filtered_raster = generic_filter(src.read(1), function=np.mean, footprint=footprint)
        else:
            filtered_raster = src.read(1)  # No filtering, use original data

    # Update profile to write float32 data type
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')

    # Save the filtered result back to a new GeoTIFF file using rasterio
    with rasterio.open(output_file, 'w', **profile) as dst:
        dst.write(filtered_raster.astype(rasterio.float32), 1)

    print(f"Filtered raster saved to: {output_file}")

def gdal_tpi(dem_file, tpi_output):
    """
    Generate a Topographic Position Index (TPI) map from a DEM using GDAL's Python bindings
    with the `compute_edges` flag enabled.

    Parameters:
    - dem_file: str, path to the input DEM file.
    - tri_output: str, path to save the generated TPI map.
    """
    # Open the DEM file
    dem_dataset = gdal.Open(dem_file)
    
    # Set options for DEM processing, including compute_edges
    options = gdal.DEMProcessingOptions(computeEdges=True)
    
    # Generate TPI using GDAL's internal function with computeEdges enabled
    gdal.DEMProcessing(tpi_output, dem_dataset, "TPI", options=options)
    print(f"TPI generated with computeEdges=True and saved to temporary file: {tpi_output}")

def gdal_tri(dem_file, tri_output):
    """
    Generate a Terrain Ruggedness Index (TRI) map from a DEM using GDAL's Python bindings
    with the `compute_edges` flag enabled.

    Parameters:
    - dem_file: str, path to the input DEM file.
    - tri_output: str, path to save the generated TRI map.
    """
    # Open the DEM file
    dem_dataset = gdal.Open(dem_file)
    
    # Set options for DEM processing, including compute_edges
    options = gdal.DEMProcessingOptions(computeEdges=True)
    
    # Generate TRI using GDAL's internal function with computeEdges enabled
    gdal.DEMProcessing(tri_output, dem_dataset, "TRI", options=options)
    print(f"TRI generated with computeEdges=True and saved to temporary file: {tri_output}")


def filtered_tpi(dem_file, final_output, radius=0):
    """
    Wrapper function to generate TPI from DEM and optionally apply a circular mean filter.

    Parameters:
    - dem_file: str, path to the input DEM file.
    - final_output: str, path to save the final TPI output (filtered or unfiltered).
    - radius: int, radius of the disk for the mean filter. If radius = 0, no filtering is applied.
    """
    # Create a temporary file for the intermediate TPI
    # Step 1: Generate TPI using GDAL's Python bindings with computeEdges enabled
    temp_tpi_path = tempfile.mktemp(suffix='.tif')
    gdal_tpi(dem_file, temp_tpi_path)

    # Step 2: Apply circular mean filter on the TPI GeoTIFF file if specified
    circular_mean_filter(temp_tpi_path, final_output, radius)

    os.remove(temp_tpi_path)
    print(f"Process complete. Final TPI saved to: {final_output}")


def filtered_tri(dem_file, final_output, radius=0):
    """
    Wrapper function to generate TRI from DEM and optionally apply a circular mean filter.

    Parameters:
    - dem_file: str, path to the input DEM file.
    - final_output: str, path to save the final TRI output (filtered or unfiltered).
    - radius: int, radius of the disk for the mean filter. If radius = 0, no filtering is applied.
    """
    # Create a temporary file for the intermediate TRI
    # Step 1: Generate TRI using GDAL's Python bindings with computeEdges enabled
    temp_tri_path = tempfile.mktemp(suffix='.tif')
    gdal_tri(dem_file, temp_tri_path)

    # Step 2: Apply circular mean filter on the TRI GeoTIFF file if specified
    circular_mean_filter(temp_tri_path, final_output, radius)

    os.remove(temp_tri_path)
    print(f"Process complete. Final TRI saved to: {final_output}")

def gdal_slope(input_dem_path, output_slope_path):
    """
    Function to calculate slope using gdaldem, with a scale factor based on the central latitude.
    
    Args:
    - input_dem_path (str): Path to the input DEM file.
    - output_slope_path (str): Path to save the output slope file.
    """
    # Function to calculate meters per degree at a given latitude
    def meters_per_degree(latitude):
        meters_per_degree_lat = 111132.92 - 559.82 * np.cos(2 * np.radians(latitude)) + 1.175 * np.cos(4 * np.radians(latitude))
        return meters_per_degree_lat

    # Open the DEM
    with rasterio.open(input_dem_path) as src:
        # Get the affine transform and DEM dimensions
        transform = src.transform
        height = src.height
        width = src.width

        # Calculate the central pixel's latitude
        center_row, center_col = height // 2, width // 2
        _, center_latitude = rasterio.transform.xy(transform, center_row, center_col, offset='center')

        # Calculate the meters per degree at the central latitude
        scale_factor = int(meters_per_degree(center_latitude))
        print(f"Central Latitude: {center_latitude}, Scale Factor (meters per degree): {scale_factor}")

    # Open the DEM with GDAL
    dem_dataset = gdal.Open(input_dem_path)

    print(scale_factor)

    # Set the options for slope calculation using the scale factor
    options = gdal.DEMProcessingOptions(scale=scale_factor, slopeFormat='degree', computeEdges=False)

    # Calculate slope using gdaldem with the specified scale factor
    gdal.DEMProcessing(output_slope_path, dem_dataset, "slope", options=options)

    print(f"Slope calculation complete. Slope raster saved to {output_slope_path}.")

def filtered_slope(dem_file, final_output, radius=0):
    """
    Wrapper function to calculate slope from DEM, apply circular mean filter, and reproject back.

    Parameters:
    - dem_file: str, path to the input DEM file.
    - final_output: str, path to save the final filtered slope output.
    - radius: int, radius of the disk for the mean filter. If radius = 0, no filtering is applied.
    """
    # Create temporary files
    temp_slope_path = tempfile.mktemp(suffix='.tif')

    # Calculate Slope using GDAL in degrees
    gdal_slope(dem_file, temp_slope_path)

    # Apply Circular Mean Filter on the Slope GeoTIFF file if specified
    circular_mean_filter(temp_slope_path, final_output, radius)

    # Remove temporary files
    os.remove(temp_slope_path)
    
    print(f"Process complete. Final filtered slope saved to: {final_output}")

### Geomorphons

In [ ]:
import os
import rasterio
import numpy as np
import whitebox as wbt  # Assuming WhiteboxTools is installed


def calculate_scale_factor(lat):
    """
    Calculate the scale factor to convert vertical units (meters) to degrees at the given latitude.
    Assumes Earth's radius is ~6371000 meters.

    Args:
        lat (float): Latitude in degrees.

    Returns:
        float: Scale factor to convert meters to degrees.
    """
    earth_radius = 6371000  # Earth's radius in meters
    scale_factor = 1 / ((2 * np.pi * earth_radius) / 360 * np.cos(np.radians(lat)))
    return scale_factor


def apply_scale_factor(dem_path, output_path, scale_factor):
    """
    Apply a scale factor to the vertical units of a DEM.

    Args:
        dem_path (str): Path to the input DEM.
        output_path (str): Path to save the scaled DEM.
        scale_factor (float): Scale factor to apply.
    """
    with rasterio.open(dem_path) as src:
        profile = src.profile
        data = src.read(1).astype(float)  # Read DEM data as float for scaling
        scaled_data = data * scale_factor  # Apply scale factor
        with rasterio.open(output_path, "w", **profile) as dst:
            dst.write(scaled_data, 1)

def generate_multiple_geomorphons(dem_path_original, L, t_list, output_path):

    scaled_dem_path = tempfile.mktemp(suffix='.tif')

    # Step 1: Calculate scale factor based on centroid latitude
    with rasterio.open(dem_path_original) as dem_src:
        bounds = dem_src.bounds
        centroid_lat = (bounds.bottom + bounds.top) / 2
        print(f"Centroid Latitude: {centroid_lat}")
        
    scale_factor = calculate_scale_factor(centroid_lat)
    print(f"Scale Factor: {scale_factor}")

    # Step 2: Apply scale factor to DEM
    apply_scale_factor(dem_path_original, scaled_dem_path, scale_factor)
    print(f"Scaled DEM saved to: {scaled_dem_path}")

    for t in t_list:

        # Step 3: Run geomorphons
        geomorphons_path = tempfile.mktemp(suffix='.tif')
        wbt_tools = wbt.WhiteboxTools()
        wbt_tools.set_verbose_mode(False)
        wbt_tools.geomorphons(
            scaled_dem_path,
            geomorphons_path,
            search=L,
            threshold=t,
            skip=5,
            forms=True,
        )
        print(f"Geomorphons output saved to: {geomorphons_path}")

        # Step 5: Align to reference raster
        aligned_to_ref_path = output_path + f'/geomorphons_L{L}_t{t}.tif'
        align_raster(geomorphons_path, reference_path, "near", "UInt32", aligned_to_ref_path)
        print(f"Aligned to reference output saved to: {aligned_to_ref_path}")
        os.remove(geomorphons_path)

    os.remove(scaled_dem_path)

## Hydrologic

In [19]:
import os
import whitebox
import rasterio
import numpy as np
import tempfile

def dw_water_occurence(roi, start, end, output_path, tile_size=32):
    dynamic_world = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start, end).filterBounds(roi)
    dynamic_world = dynamic_world.map(mask_edges)
    day_mosaics = daily_mosaics(dynamic_world)

    def apply_filter(image):
        return image.eq(0)

    water = day_mosaics.select('label').map(apply_filter)
    water_occ = water.sum().divide(water.count())

    geemap.download_ee_image(water_occ,
                     filename=output_path, 
                     region=roi, 
                     crs='EPSG:4326', 
                     resampling='near',
                     dtype='float32',
                     scale=10,
                     max_tile_size=tile_size)

def get_prob(image):
    image.select('water').add(image.select('flooded_vegetation'))

    return image.select('water').add(image.select('flooded_vegetation'))
    
def dw_water_prob(roi, start, end, output_path, tile_size=32):
    dynamic_world = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start, end).filterBounds(roi)
    dynamic_world = dynamic_world.map(mask_edges)
   
    day_mosaics = daily_mosaics(dynamic_world)

    water = day_mosaics.map(get_prob)
 
    water_features = extract_features_gee(water)

    geemap.download_ee_image(water_features,
                     filename=output_path, 
                     region=roi, 
                     crs='EPSG:4326', 
                     resampling='near',
                     dtype='float32',
                     scale=10,
                     max_tile_size=tile_size)

    
def th_hydro_dem(dem, hydro_dem):

    # Define Terrahidro path
    th_path = 'C:/Users/gabro/TerraHidro-5.2.0'

    # Execute remove pits
    cmd = rf'cd\ && cd {th_path} && th removepits "{dem}" "{hydro_dem}"'
    flag = os.system(cmd)

    if flag != 0:
        print('Error executing th removepits command.')
    else:
        print('Hydro DEM done.')

def th_d8(hydro_dem, hydro_dem_d8):

     # Define Terrahidro path
    th_path = 'C:/Users/gabro/TerraHidro-5.2.0'

    # Execute d8
    cmd = rf'cd\ && cd {th_path} && th d8 "{hydro_dem}" "{hydro_dem_d8}"'
    flag = os.system(cmd)

    if flag != 0:
        print('Error executing th d8 command.')
    else:
        print('Calculating d8 done.')

def th_contributing_area(hydro_dem_d8, hydro_dem_ca):

     # Define Terrahidro path
    th_path = 'C:/Users/gabro/TerraHidro-5.2.0'

    # Execute d8ca
    cmd = rf'cd\ && cd {th_path} && th d8ca "{hydro_dem_d8}" "{hydro_dem_ca}"'
    flag = os.system(cmd)

    if flag != 0:
        print('Error executing d8ca command.')
    else:
        print('Calculating th dbca done.')

def th_drainage(hydro_dem_ca, hydro_dem_drainage, ca_threshold):

     # Define Terrahidro path
    th_path = 'C:/Users/gabro/TerraHidro-5.2.0'   

    # Execute d8drainage
    cmd = rf'cd\ && cd {th_path} && th d8drainage "{hydro_dem_ca}" "{hydro_dem_drainage}" {ca_threshold}'
    flag = os.system(cmd)

    if flag != 0:
        print('Error executing th d8drainage command.')
    else:
        print(f'Drainage extracted: {hydro_dem_drainage}')

def convert_ldd_geotiff(input_ldd_geotiff_path, output_ldd_geotiff_path=None):
    """
    Convert LDD values in a GeoTIFF file using a predefined mapping.

    Args:
    input_ldd_geotiff_path (str): Path to the input LDD GeoTIFF file.
    output_ldd_geotiff_path (str, optional): Path to save the converted LDD GeoTIFF file. 
                                             If None, no output file is saved.

    Returns:
    np.ndarray: The converted LDD array.
    """
    # Open the GeoTIFF using rasterio
    with rasterio.open(input_ldd_geotiff_path) as dataset:
        # Read the data from the first band
        ldd_data = dataset.read(1)
        crs = dataset.crs
        transform = dataset.transform

    # Ensure we handle only the LDD values [1, 2, 4, 8, 16, 32, 64, 128, 0]
    valid_ldd_values = {1, 2, 4, 8, 16, 32, 64, 128, 0}

    # Define the mapping from the origin coding to the target coding
    ldd_conversion = {
        1: 2,     # East -> Southeast
        2: 4,     # Southeast -> South
        4: 8,     # South -> Southwest
        8: 16,    # Southwest -> West
        16: 32,   # West -> Northwest
        32: 64,   # Northwest -> North
        64: 128,  # North -> Northeast
        128: 1,   # Northeast -> East
        0: 0      # No flow remains unchanged
    }

    # Apply the conversion only for valid LDD values, keeping all other values unchanged
    def convert_ldd(value):
        if value in valid_ldd_values:
            return ldd_conversion.get(value, value)
        else:
            return value

    # Vectorize the conversion function
    converted_ldd = np.vectorize(convert_ldd)(ldd_data)

    # Replace any values that are not part of the valid LDD set with 0
    valid_values = [0, 1, 2, 4, 8, 16, 32, 64, 128]
    converted_ldd = np.where(np.isin(converted_ldd, valid_values), converted_ldd, 0)

    # Optionally, save the converted LDD to a new GeoTIFF if output path is provided
    if output_ldd_geotiff_path:
        with rasterio.open(
            output_ldd_geotiff_path,
            'w',
            driver='GTiff',
            height=converted_ldd.shape[0],
            width=converted_ldd.shape[1],
            count=1,
            dtype=converted_ldd.dtype,
            crs=crs,
            transform=transform,
        ) as new_dataset:
            new_dataset.write(converted_ldd, 1)

    return converted_ldd

def max_strahler_order(dem_path, ord_strahler_max_path):

    # aux files paths (they are deleted at the end of the process)
    hydro_dem_path = tempfile.mktemp(suffix='.tif')
    hydro_dem_d8_path = tempfile.mktemp(suffix='.tif')
    hydro_dem_ca_path = tempfile.mktemp(suffix='.tif')
    ca_gt_zero_path = tempfile.mktemp(suffix='.tif')
    hydro_dem_d8_convert_path = tempfile.mktemp(suffix='.tif') 
    
    # generate ldd and drainage (ca > 0)
    th_hydro_dem(dem_path, hydro_dem_path)
    th_d8(hydro_dem_path, hydro_dem_d8_path)
    th_contributing_area(hydro_dem_d8_path, hydro_dem_ca_path)
    th_drainage(hydro_dem_ca_path, ca_gt_zero_path, 0)
    
    # convert ldd coding from th to whitebox
    convert_ldd_geotiff(hydro_dem_d8_path, hydro_dem_d8_convert_path)
    
    # generate max strahler order
    wbt = whitebox.WhiteboxTools()
    wbt.set_verbose_mode(False)
    wbt.strahler_stream_order(hydro_dem_d8_convert_path, ca_gt_zero_path, ord_strahler_max_path)
    
    # remove aux files
    os.remove(hydro_dem_path)
    os.remove(hydro_dem_d8_path)
    os.remove(hydro_dem_ca_path)
    os.remove(ca_gt_zero_path)
    os.remove(hydro_dem_d8_convert_path)

def decompose_strahler(drainage):
    # Open the input raster
    with rasterio.open(drainage, 'r') as src:
        # Read the raster data
        drainage_data = src.read(1)

        # Define the Strahler thresholds
        thresholds = [3, 4, 5, 6, 7, 8, 9]

        for threshold in thresholds:
            # Create a new binary raster based on the threshold
            # Pixels with Strahler order >= threshold will be set to 1, others to 0
            
            filtered_data = (drainage_data >= threshold).astype(src.dtypes[0])

            # Define the output raster path
            prefix = drainage.split('.')[0]
            suffix = drainage.split('.')[1]
            output_raster_path = f'{prefix}_{threshold}.{suffix}'

            # Write the filtered data to the new raster file
            with rasterio.open(output_raster_path, 'w', driver='GTiff',
                               height=src.height, width=src.width,
                               count=1, dtype=filtered_data.dtype,
                               crs=src.crs, transform=src.transform) as dst:
                dst.write(filtered_data.astype(rasterio.uint8), 1)

    # Return list of paths
    return [f'{prefix}_{threshold}.{suffix}' for threshold in thresholds]

def merge_waterbodies_strahler(waterbodies, strahler_drainage, output_path):

    with rasterio.open(waterbodies) as src1, rasterio.open(strahler_drainage) as src2:

        # Read the data from the rasters
        raster1_data = src1.read(1)
        raster2_data = src2.read(1)

        # Create an empty array for the new raster with the same shape as raster1
        newraster_data = raster1_data.copy()

        # Apply the conditions
        newraster_data = np.where(raster1_data > 0, 99*raster1_data, raster2_data)

        # Write the new raster using the profile of raster1
        with rasterio.open(output_path, 'w', **src1.profile) as dst:
            dst.write(newraster_data, 1)  

def decompose_waterbodies_strahler(drainage):
    # Open the input raster
    with rasterio.open(drainage, 'r') as src:
        # Read the raster data
        drainage_data = src.read(1)

        # Define the Strahler thresholds
        thresholds = [3, 4, 5, 6, 7, 8, 9]

        for threshold in thresholds:
            # Create a new binary raster based on the threshold
            # Pixels with Strahler order >= threshold will be set to 1, others to 0
            # if threshold == 99:
            #     drainage_data =  np.where(drainage_data == 99, drainage_data, 0)
            
            filtered_data = (drainage_data >= threshold).astype(src.dtypes[0])

            # Define the output raster path
            prefix = drainage.split('.')[0]
            suffix = drainage.split('.')[1]
            output_raster_path = f'{prefix}_{threshold}.{suffix}'

            # Write the filtered data to the new raster file
            with rasterio.open(output_raster_path, 'w', driver='GTiff',
                               height=src.height, width=src.width,
                               count=1, dtype=filtered_data.dtype,
                               crs=src.crs, transform=src.transform) as dst:
                dst.write(filtered_data.astype(rasterio.uint8), 1)

    # Return list of paths
    return [f'{prefix}_{threshold}.{suffix}' for threshold in thresholds]



def th_hand(original_dem, hydro_dem_d8, drainage, hand):

    # Define Terrahidro path
    th_path = 'C:/Users/gabro/TerraHidro-5.2.0'

    # Execute HAND
    cmd = rf'cd\ && cd "{th_path}" && th hand "{original_dem}" "{hydro_dem_d8}" "{drainage}" "{hand}"'
    flag = os.system(cmd)

    if flag != 0:
        print('Error executing hand command.')
    else:
        print(f'HAND extracted: {hand}')

def get_next_pixel(row, col, direction):
    # Input LDD codification (we use terrahidro one)    
    E  = 1
    SE = 2
    S  = 4
    SW = 8
    W  = 16
    NW = 32
    N  = 64
    NE = 128

    # Get position of next pixel according to LDD
    if direction == E:
        return row, col + 1
    elif direction == SE:
        return row + 1, col + 1
    elif direction == S:
        return row + 1, col
    elif direction == SW: 
        return row + 1, col - 1
    elif direction == W:
        return row, col - 1
    elif direction == NW:
        return row - 1, col - 1
    elif direction == N:
        return row - 1, col
    elif direction == NE:
        return row - 1, col + 1
    elif direction == 0:
        return row, col
    else:
        print(f'Error: Flow direction value not recognized: {direction}!')
        exit()         

def pixel_distance(i1, j1, i2, j2):
    import math
    return math.sqrt((i2 - i1)**2 + (j2 - j1)**2)

def hdnd(ldd_path, drain_path, hdnd_path):

    # Read the rasters
    with rasterio.open(ldd_path) as src:
        ldd = src.read(1)
    with rasterio.open(drain_path) as src:
        res = src.res[0]
        drain = src.read(1)

    # Creating initial hdnd raster
    hdnd = np.zeros(ldd.shape, dtype='float64')
    hdnd = np.where(ldd==0, -np.inf, hdnd)

    # Get rows and cols
    rows, cols = hdnd.shape

    # Iterate hdnd pixels
    for row in range(rows):
        for col in range(cols):

            if hdnd[row, col] == 0 and drain[row, col] == 0:

                # Follow the flow path until a drain value !=0 is found
                d = 0
                current_row, current_col = row, col
                while drain[current_row, current_col] == 0 and hdnd[current_row, current_col] == 0:
                    flow = ldd[current_row, current_col]
                    next_row, next_col = get_next_pixel(current_row, current_col, flow)
                    d = d + pixel_distance(current_row, current_col, next_row, next_col)*res + hdnd[next_row, next_col]
                    current_row, current_col = next_row, next_col


                # Save d value to D and restart d
                D = d
                d = 0

                # Follow flow path again setting the values
                current_row, current_col = row, col
                while drain[current_row, current_col] == 0 and hdnd[current_row, current_col] == 0:
                    flow = ldd[current_row, current_col]
                    hdnd[current_row, current_col] = D - d
                    next_row, next_col = get_next_pixel(current_row, current_col, flow)
                    d = d + pixel_distance(current_row, current_col, next_row, next_col)*res 
                    current_row, current_col = next_row, next_col

    # Save the eca raster
    with rasterio.open(ldd_path) as src:  # Using ldd file to get the metadata
        profile = src.profile
        profile.update(dtype=rasterio.float32, nodata=-np.inf, count=1, compress='lzw')

        # Write eca
        with rasterio.open(hdnd_path, 'w', **profile) as dst:
            dst.write(hdnd.astype(rasterio.float32), 1)

    print(f'HDND extracted: {hdnd_path}')

def ee_gsw_max_extent(area, ref_dem, output_path):
    # Get GSW data
    gswImage = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')

    # Select max_extent
    maxExtent = gswImage.select('max_extent')

    # Converting nodata to 0
    maxExtent = maxExtent.where(maxExtent.neq(1),0)
    maxExtent = maxExtent.unmask(0)

    # Casting to integer
    dict = {'max_extent'  : 'int8'}
    maxExtent = maxExtent.cast(dict)

    # Set output name
    output = f'{output_path}/gsw_max_extent.tif'

    # Reprojecting to match the DEM
    proj = ref_dem.projection()
    maxExtent = maxExtent.reproject(crs=proj)

    # Download max extent
    geemap.ee_export_image(maxExtent, filename=output, region=area)


    return output

def gsw_occurrence(area, output_path):
    # Get GSW data

    import tempfile

    tempfile = tempfile.mktemp(suffix='.tif')
    gswImage = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')

    # Download gsw data
    geemap.download_ee_image(gswImage.select(0),
                 filename=tempfile, 
                 region=roi, 
                 crs='EPSG:4326', 
                 resampling='near',
                 dtype='float32',
                 unmask_value=0,
                 scale=10)

    with rasterio.open(tempfile) as src: 
        profile = src.profile
        data =  src.read(1)
        profile.update(nodata=0)

    # Write
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(data/100, 1)  


    return output_path
    
def dw_urban_mask(roi, start, end, min_holes, min_objects, output_path):

    import tempfile
    from skimage import morphology

    dynamic_world = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start, end).filterBounds(roi)
    day_mosaics = daily_mosaics(dynamic_world)

    def apply_filter(image):
        return image.eq(6)

    urban = day_mosaics.select('label').map(apply_filter).mode()

    raw_urban = tempfile.mktemp(suffix='.tif')

    geemap.download_ee_image(urban,
                     filename=raw_urban, 
                     region=roi, 
                     crs='EPSG:4326', 
                     resampling='near',
                     dtype='float32',
                     scale=10,
                     max_tile_size=5)

    with rasterio.open(raw_urban) as src: 
        profile = src.profile
        data =  src.read(1).astype(np.uint8)
        profile.update(dtype=rasterio.uint8, count=1, compress='lzw')
        profile.update(nodata=0)

    data_filtered_holes = morphology.remove_small_holes(data, min_holes, connectivity=8)
    data_filtered_objects = morphology.remove_small_objects(data_filtered_holes, min_objects, connectivity=8)

    # Write
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(data_filtered_objects, 1)   

    os.remove(raw_urban)

# Feature Extraction

## GEE API Auth

In [20]:
ee.Authenticate() 
ee.Initialize()

Enter verification code:  4/1AVMBsJgrwaS5nyqcVJYGdxzjF3kiRTRg4k2TrVOCk_tOX9b8vMV7IS9hVrs



Successfully saved authorization token.


## Area shp to GEE

In [21]:
gdf = gpd.read_file(shapefile_path).to_crs(epsg=4326)
roi = geemap.gdf_to_ee(gdf).first().geometry()
Map = geemap.Map()
Map.addLayer(roi)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Sentinel-2

In [23]:
# Create s2 collection paired with s2cloudless
s2_raw = get_s2_sr_cld_col(roi, START_DATE, END_DATE)

# Add cloud mask based on the parameters
s2_raw = s2_raw.map(add_cld_shdw_mask)

# Check cloud masking result for tunning the parameters
#display_cloud_layers(s2_raw, roi)

In [19]:
# Filter the collection
filtered_collection = filter_images_by_band_names(s2_raw, required_bands)

# Get the size of the filtered collection
print("Number of images in the filtered collection:", filtered_collection.size().getInfo())

s2_raw = filtered_collection

Number of images in the filtered collection: 59


In [24]:
# Apply mean composite to all same day images
s2_raw = daily_images_mean(s2_raw)

# Apply cloud masking
s2 = s2_raw.map(apply_cld_shdw_mask)

# Filtering bands
s2_raw = s2_raw.select(['B2', 'B3', 'B4', 'B8'])
s2 = s2.select(['B2', 'B3', 'B4', 'B8'])

# Getting image dates and platforms 
s2_dates = list(s2.aggregate_array('system:index').getInfo())

In [28]:
# Calculate EVI and NDWI time series
s2 = s2.map(getEVI).map(getNDWI)

# Filtering only indexes
s2_indexes = s2.select(['EVI', 'NDWI'])
s2_indexes = s2_indexes.map(mask_edges)
s2_indexes = daily_images_mean(s2_indexes)

# Extracting features from indexes
s2_features = extract_features_gee(s2_indexes)

# Create output dir 
if not os.path.exists(features_path):
    os.mkdir(features_path)

# Download s2_features
s2_features_path = f'{features_path}/s2_features.tif'
geemap.download_ee_image(s2_features, 
                         filename=s2_features_path, 
                         region=roi, 
                         crs='EPSG:4326', 
                         scale=10,
                         max_tile_size=10)

## Sentinel-1

In [55]:
# Creating sentinel-1 collection
s1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
           .filterBounds(roi)
           # .filter(ee.Filter.contains('.geo', roi))
           .filterDate(ee.Date(start_date),ee.Date(end_date))
           .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
           .filter(ee.Filter.eq('instrumentMode', 'IW'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
           .select(["VV","VH"])
           .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
           .sort('date'))

# # Co-registering the images to reference
# proj = ref.projection()
# s1 = s1.map(lambda img: img.reproject(crs=proj))

# Convert multiple daily images into mean composite


def mask_edges(image):
    
    # Get the valid pixel mask
    valid_mask = image.mask()
    
    # Iteratively erode the valid mask (adjust iterations as needed)
    eroded_mask = valid_mask.focal_min(iterations=50) # Adjust the number of iterations
    
    # Apply the eroded mask to the image
    return image.updateMask(eroded_mask)


# Apply the mask eroding function to the collection
s1 = s1.map(mask_edges)

s1 = daily_images_mean(s1)

# Getting image dates and platform
s1_dates = list(s1.aggregate_array('date').getInfo())

#Download s1 features
s1_features_path = f'{features_path}/s1_features.tif'
geemap.download_ee_image(s1_features, 
                         filename=s1_features_path, 
                         region=roi, 
                         crs='EPSG:4326', 
                         scale=10,
                         max_tile_size=5)

## PALSAR yearly composites

In [58]:
col = ee.ImageCollection("JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH").filterBounds(roi).select(["HH","HV"])

# Getting band names (assuming that are equal for all images)
bandnames = col.first().bandNames().getInfo()

# Calculate mean, std and percentiles
col_mean = col.mean().rename([f'{bandname}_mean' for bandname in bandnames])
col_std = col.reduce(ee.Reducer.stdDev()).rename([f'{bandname}_std' for bandname in bandnames])
col_percentiles = col.reduce(ee.Reducer.percentile([5, 95], ['05','95']))

# Stack them and sort alphabetically
col_features = col_mean.addBands([col_std, col_percentiles])
sorted_names = col_features.bandNames().sort()
col_features = col_features.select(sorted_names)

p_features_path = f'{features_path}/palsar_features.tif'
geemap.download_ee_image(col_features, 
                         filename=p_features_path, 
                         region=roi, 
                         crs='EPSG:4326', 
                         scale=10,
                         max_tile_size=20)


## Morphometric

In [60]:
reference_path =  s1_features_path
# Generate TPI, TRI and Slope - Set the desired radius for the posterior smoothing
for r in [0,1,2,3,4,5,6]: 

    # Generate features for the extended DEM
    output_full_tpi =  f'{aux_path}/mean_full_tpi_{r}.tif'
    output_full_tri =  f'{aux_path}/mean_full_tri_{r}.tif'
    output_full_slope = f'{aux_path}/mean_full_slope_{r}.tif'
    filtered_tri(dem_path, output_full_tri, radius=r)
    filtered_tpi(dem_path,output_full_tpi, radius=r)
    filtered_slope(dem_path, output_full_slope, radius=r)

    # Align features to s2 features (resample and clip)
    output_tpi =  f'{features_path}/mean_tpi_{r}.tif'
    output_tri =  f'{features_path}/mean_tri_{r}.tif'
    output_slope = f'{features_path}/mean_slope_{r}.tif'
    align_raster(output_full_tpi, reference_path,  'bilinear', 'Float32', output_tpi)
    align_raster(output_full_tri, reference_path, 'bilinear', 'Float32', output_tri)
    align_raster(output_full_slope, reference_path, 'bilinear', 'Float32', output_slope)

TRI generated with computeEdges=True and saved to temporary file: C:\Users\gabro\AppData\Local\Temp\tmpq7a36qn_.tif
Filtered raster saved to: D:/thesis_areas/area_06/auxiliar/mean_full_tri_0.tif
Process complete. Final TRI saved to: D:/thesis_areas/area_06/auxiliar/mean_full_tri_0.tif
TPI generated with computeEdges=True and saved to temporary file: C:\Users\gabro\AppData\Local\Temp\tmpnkftptv4.tif
Filtered raster saved to: D:/thesis_areas/area_06/auxiliar/mean_full_tpi_0.tif
Process complete. Final TPI saved to: D:/thesis_areas/area_06/auxiliar/mean_full_tpi_0.tif
Central Latitude: -20.74999999988889, Scale Factor (meters per degree): 110713
110713
Slope calculation complete. Slope raster saved to C:\Users\gabro\AppData\Local\Temp\tmpqmxw0whb.tif.
Filtered raster saved to: D:/thesis_areas/area_06/auxiliar/mean_full_slope_0.tif
Process complete. Final filtered slope saved to: D:/thesis_areas/area_06/auxiliar/mean_full_slope_0.tif
gdalwarp -t_srs EPSG:4326 -te -44.2500227489151 -20.8750

In [30]:
# Geomorphons
reference_path = f"D:/thesis_areas/area_0{i}/features/s1_features.tif"
dem_path_original = f"D:/thesis_areas/area_0{i}/new_corrected_dem.tif"
L= 100
t_list = [0,1,2,3,4,5]
generate_multiple_geomorphons(dem_path_original, L, t_list, f"D:/thesis_areas/area_0{i}/features/")




## Hydrologic

### HAND and HDND

In [63]:
# DEM hydro conditioning
hydro_dem_path =  f'{aux_path}/hydro_dem.tif'
th_hydro_dem(dem_path, hydro_dem_path)

# Generate d8 for the hydro DEM
hydro_dem_d8_path = f'{aux_path}/hydro_dem_d8.tif'
th_d8(hydro_dem_path, hydro_dem_d8_path)

# Generate contributing area for the hydro DEM
hydro_dem_ca_path = f'{aux_path}/hydro_dem_ca.tif'

# Max Strahler Order Extraction
max_strahler_ord_path = f'{aux_path}/max_strahler_ord.tif'
max_strahler_order(dem_path, max_strahler_ord_path)

# Decompose merged strahler drainage by orders >= 2, 3, 4, 5
# The decompose function return the list of paths of the generated files
decomposed_strahler_path_list = decompose_strahler(max_strahler_ord_path)

# Merge waterbodies with strahler drainage
waterbodies_max_strahler_ord_path =   f'{aux_path}/waterbodies_max_strahler_ord.tif'
merge_waterbodies_strahler(waterbodies_path, max_strahler_ord_path, waterbodies_max_strahler_ord_path)

# Decompose merged strahler drainage by orders >= 2, 3, 4, 5 and 99 (99 is only the water bodies)
# The decompose function return the list of paths of the generated files
decomposed_waterbodies_strahler_path_list = decompose_waterbodies_strahler(waterbodies_max_strahler_ord_path)

# #Generate HANDs for the decomposed strahler drainages
for i in range(len(decomposed_waterbodies_strahler_path_list)):
    path = decomposed_waterbodies_strahler_path_list[i]
    output_full = f'{aux_path}/' + os.path.basename(decomposed_waterbodies_strahler_path_list[i]).split(".")[0] + "_hand.tif"
    th_hand(dem_path, hydro_dem_d8_path, path, output_full)
    output_align = f'{features_path}/' + os.path.basename(decomposed_waterbodies_strahler_path_list[i]).split(".")[0] + "_hand.tif"
    align_raster(output_full, reference_path, 'bilinear','Float32', output_align)

#Generate HDNDs for the decomposed strahler drainages
for i in range(len(decomposed_waterbodies_strahler_path_list)):
    path = decomposed_waterbodies_strahler_path_list[i]
    output_full = f'{aux_path}/' + os.path.basename(decomposed_waterbodies_strahler_path_list[i]).split(".")[0] + "_hdnd.tif"
    hdnd(hydro_dem_d8_path, path, output_full)
    output_align = f'{features_path}/' + os.path.basename(decomposed_waterbodies_strahler_path_list[i]).split(".")[0] + "_hdnd.tif"
    align_raster(output_full, reference_path, 'bilinear','Float32', output_align)

# Donwload GSW occurrence
output_full = f'{aux_path}/gsw_occurrence.tif'
gsw_occurrence(roi, output_full)
output_align = f'{features_path}/gsw_occurrence.tif'
align_raster(output_full, reference_path, 'nearest', 'Float32', output_align)

# Download dynamic world urban areas mask (to be used only in the sampling notebook)
urban_areas_path = f'{aux_path}/urban_areas.tif'
dw_urban_mask(roi, '2023-01-01', '2023-12-31', 1000, 1000, urban_areas_path)

Hydro DEM done.
Calculating d8 done.
Hydro DEM done.
Calculating d8 done.
Calculating th dbca done.
Drainage extracted: C:\Users\gabro\AppData\Local\Temp\tmpb06xo1p1.tif
HAND extracted: D:/thesis_areas/area_06/auxiliar/waterbodies_max_strahler_ord_3_hand.tif
gdalwarp -t_srs EPSG:4326 -te -44.2500227489151 -20.87505057236944 -43.99993177381622 -20.624959597270564 -tr 8.983152841195215e-05 8.983152841195215e-05 "D:/thesis_areas/area_06/auxiliar/waterbodies_max_strahler_ord_3_hand.tif" "D:/thesis_areas/area_06/features/waterbodies_max_strahler_ord_3_hand.tif" -r bilinear -ot Float32 -overwrite
HAND extracted: D:/thesis_areas/area_06/auxiliar/waterbodies_max_strahler_ord_4_hand.tif
gdalwarp -t_srs EPSG:4326 -te -44.2500227489151 -20.87505057236944 -43.99993177381622 -20.624959597270564 -tr 8.983152841195215e-05 8.983152841195215e-05 "D:/thesis_areas/area_06/auxiliar/waterbodies_max_strahler_ord_4_hand.tif" "D:/thesis_areas/area_06/features/waterbodies_max_strahler_ord_4_hand.tif" -r biline

tmpplgcw1tg.tif: |                                                    | 0.00/31.0M (raw) [  0.0%] in 00:00 (et…

gdalwarp -t_srs EPSG:4326 -te -44.2500227489151 -20.87505057236944 -43.99993177381622 -20.624959597270564 -tr 8.983152841195215e-05 8.983152841195215e-05 "D:/thesis_areas/area_06/auxiliar/gsw_occurrence.tif" "D:/thesis_areas/area_06/features/gsw_occurrence.tif" -r nearest -ot Float32 -overwrite


tmp0qvj0vno.tif: |                                                    | 0.00/31.0M (raw) [  0.0%] in 00:00 (et…

## GSW ocurrence and DW probability

In [32]:
# Donwload GSW occurrence
output_full = f'{aux_path}/gsw_occurrence.tif'
gsw_occurrence(roi, output_full)
output_align = f'{features_path}/gsw_occurrence.tif'
align_raster(output_full, reference_path, 'nearest', 'Float32', output_align)

# Download Dynamic world water and flooded vegetation prob
dw_water_prob(roi, START_DATE, END_DATE, f"{features_path}/dw_prob.tif", tile_size=5)

## Contextual Variations

### Morphometric

In [36]:
import rasterio
import numpy as np
from scipy.ndimage import uniform_filter
import os

# Function to create the features
def create_filtered_stack(input_path, sizes, output_path=None):
    """
    Creates a 3D raster stack with the original band and its uniform-filtered versions.

    Args:
        input_path (str): Path to the single-band input raster.
        sizes (list): List of filter sizes to apply.
        output_path (str, optional): Path to save the output raster. If None, the output file
                                      will be named based on the input file.

    Returns:
        str: Path to the output raster.
    """
    # Open the input raster
    with rasterio.open(input_path) as src:
        # Ensure the raster is single-band
        if src.count != 1:
            raise ValueError("Input raster must have only one band.")

        # Read the single band
        original_data = src.read(1)
        profile = src.profile

        # Update profile for a multi-band output
        profile.update(count=len(sizes) + 1, dtype='float32')

        # Initialize stack with original data
        stack = [original_data.astype('float32')]

        # Apply uniform filter for each size and add to stack
        for size in sizes:
            filtered = uniform_filter(original_data, size=size, mode='nearest')
            stack.append(filtered)

        # Convert stack to numpy array (bands, rows, cols)
        stack = np.array(stack)

        # Generate band names
        band_descriptions = ["0x0"] + [f"{size}x{size}" for size in sizes]

        # Define the output file path
        if output_path is None:
            directory, filename = os.path.split(input_path)
            name, ext = os.path.splitext(filename)
            output_filename = f"{name}_stack{ext}"
            output_path = os.path.join(directory, output_filename)

        # Write the stack to the output file
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(stack)
            dst.descriptions = tuple(band_descriptions)

    return output_path

# Generating Contextual Morphometric Features
filter_sizes = [11,21,41,61,81,101] 

input_raster = f"Z:/thesis_areas/area_0{i}/features/mean_slope_0.tif"   
output_raster = create_filtered_stack(input_raster, filter_sizes)
print(f"Filtered stack saved to: {output_raster}")

input_raster = f"Z:/thesis_areas/area_0{i}/features/mean_tpi_0.tif"  
output_raster = create_filtered_stack(input_raster, filter_sizes)
print(f"Filtered stack saved to: {output_raster}")

input_raster = f"Z:/thesis_areas/area_0{i}/features/mean_tri_0.tif"  
output_raster = create_filtered_stack(input_raster, filter_sizes)
print(f"Filtered stack saved to: {output_raster}")


### Temporal

In [37]:
import rasterio
import numpy as np
from scipy.ndimage import uniform_filter
import os

def apply_uniform_filter(input_path, size):
    """
    Applies a uniform filter to a 3D raster feature stack and saves the result to a new file.

    Args:
        input_path (str): Path to the input raster.
        size (int): Size of the uniform filter.

    Returns:
        str: Path to the output raster.
    """
    # Open the input raster
    with rasterio.open(input_path) as src:
        # Read all bands into a numpy array
        data = src.read()
        profile = src.profile

        # Get band descriptions
        band_descriptions = src.descriptions

        # Ensure the output data type remains consistent
        profile.update(dtype='float32')

        # Apply the uniform filter to each band
        filtered_data = np.zeros_like(data, dtype=np.float32)
        for band in range(data.shape[0]):
            filtered_data[band] = uniform_filter(data[band], size=size, mode='nearest')

        # Create the output file name
        directory, filename = os.path.split(input_path)
        name, ext = os.path.splitext(filename)
        output_filename = f"{name}_mean_{size}{ext}"
        output_path = os.path.join(directory, output_filename)

        # Write the filtered data to the output file
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(filtered_data)
            
            # Set the band descriptions
            dst.descriptions = band_descriptions

    return output_path


# Generating contextual variations
sizes = [11,21,41,61,81,101]

# Sentinel-2
for size in sizes:
    input_raster =   f"D:/thesis_areas/area_0{i}/features/s2_features.tif"# Replace with your input raster path
    output_raster = apply_uniform_filter(input_raster, size)
    print(f"Filtered raster saved to: {output_raster}")

# Sentinel-1
for size in sizes:
    input_raster =   f"D:/thesis_areas/area_0{i}/features/s1_features.tif"# Replace with your input raster path
    output_raster = apply_uniform_filter(input_raster, size)
    print(f"Filtered raster saved to: {output_raster}")
    
# Palsar
for size in sizes:
    input_raster =   f"D:/thesis_areas/area_0{i}/features/palsar_features.tif"# Replace with your input raster path
    output_raster = apply_uniform_filter(input_raster, size)
    print(f"Filtered raster saved to: {output_raster}")

## Other

### Urban areas mask

In [ ]:
# Download dynamic world urban areas mask (to be used only in the sampling notebook)
urban_areas_path = f'{aux_path}/urban_areas.tif'
dw_urban_mask(roi, '2023-01-01', '2023-12-31', 1000, 1000, urban_areas_path)

### GWL_FSC30 - Zhang et al (2023) 

In [40]:
# Download GWL_FSC30 dara for comparison with the result
col = ee.ImageCollection("projects/sat-io/open-datasets/GWL_FCS30")

# Download binarized version of the
zhang_features_path = f'{features_path}/GWL_FCS30_2022.tif'
geemap.download_ee_image(col.filterDate('2022-01-01', '2022-12-31').first().gt(180), 
                         filename=zhang_features_path, 
                         region=roi, 
                         crs='EPSG:4326', 
                         scale=10)
